## Homework: Multilingual Embedding-based Machine Translation (6 points)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**In this homework** **<font color='red'>YOU</font>** will make a machine translation system without using parallel corpora, alignment, attention, 100500-depth super-cool recurrent neural networks and all that kind of superstuff.

But even without parallel corpora, this system can be good enough (hopefully).

For our system we choose two kindred Slavic languages: Ukrainian and Russian.

### Feel the difference!

(_синій кіт_ vs. _синій кит_)

![blue_cat_blue_whale.png](https://github.com/yandexdataschool/nlp_course/raw/master/resources/blue_cat_blue_whale.png)

### Fragment of the Swadesh list for some Slavic languages

The Swadesh list is a lexicostatistical tool. It's named after American linguist Morris Swadesh and contains basic lexis. These lists are used to define subgroupings of languages and their relatedness.

So we can see some kind of word invariance for different Slavic languages.


| Russian         | Belorussian              | Ukrainian               | Polish             | Czech                         | Bulgarian            |
|-----------------|--------------------------|-------------------------|--------------------|-------------------------------|-----------------------|
| женщина         | жанчына, кабета, баба    | жінка                   | kobieta            | žena                          | жена                  |
| мужчина         | мужчына                  | чоловік, мужчина        | mężczyzna          | muž                           | мъж                   |
| человек         | чалавек                  | людина, чоловік         | człowiek           | člověk                        | човек                 |
| ребёнок, дитя   | дзіця, дзіцёнак, немаўля | дитина, дитя            | dziecko            | dítě                          | дете                  |
| жена            | жонка                    | дружина, жінка          | żona               | žena, manželka, choť          | съпруга, жена         |
| муж             | муж, гаспадар            | чоловiк, муж            | mąż                | muž, manžel, choť             | съпруг, мъж           |
| мать, мама      | маці, матка              | мати, матір, неня, мама | matka              | matka, máma, 'стар.' mateř    | майка                 |
| отец, тятя      | бацька, тата             | батько, тато, татусь    | ojciec             | otec                          | баща, татко           |
| много           | шмат, багата             | багато                  | wiele              | mnoho, hodně                  | много                 |
| несколько       | некалькі, колькі         | декілька, кілька        | kilka              | několik, pár, trocha          | няколко               |
| другой, иной    | іншы                     | інший                   | inny               | druhý, jiný                   | друг                  |
| зверь, животное | жывёла, звер, істота     | тварина, звір           | zwierzę            | zvíře                         | животно               |
| рыба            | рыба                     | риба                    | ryba               | ryba                          | риба                  |
| птица           | птушка                   | птах, птиця             | ptak               | pták                          | птица                 |
| собака, пёс     | сабака                   | собака, пес             | pies               | pes                           | куче, пес             |
| вошь            | вош                      | воша                    | wesz               | veš                           | въшка                 |
| змея, гад       | змяя                     | змія, гад               | wąż                | had                           | змия                  |
| червь, червяк   | чарвяк                   | хробак, черв'як         | robak              | červ                          | червей                |
| дерево          | дрэва                    | дерево                  | drzewo             | strom, dřevo                  | дърво                 |
| лес             | лес                      | ліс                     | las                | les                           | гора, лес             |
| палка           | кій, палка               | палиця                  | patyk, pręt, pałka | hůl, klacek, prut, kůl, pálka | палка, пръчка, бастун |

But the context distribution of these languages demonstrates even more invariance. And we can use this fact for our purposes.

## Data

In [2]:
pip install gensim

In [3]:
import gensim
import numpy as np
from gensim.models import KeyedVectors

Download embeddings here:
* [cc.uk.300.vec.zip](https://yadi.sk/d/9CAeNsJiInoyUA)
* [cc.ru.300.vec.zip](https://yadi.sk/d/3yG0-M4M8fypeQ)

Load embeddings for Ukrainian and Russian.

In [4]:
uk_emb = KeyedVectors.load_word2vec_format("/content/drive/MyDrive/cc.uk.300.vec")

In [5]:
ru_emb = KeyedVectors.load_word2vec_format("/content/drive/MyDrive/cc.ru.300.vec")

In [6]:
ru_emb.most_similar([ru_emb["август"]], topn=10)

[('август', 1.0000001192092896),
 ('июль', 0.9383152723312378),
 ('сентябрь', 0.9240029454231262),
 ('июнь', 0.9222574830055237),
 ('октябрь', 0.9095539450645447),
 ('ноябрь', 0.8930036425590515),
 ('апрель', 0.8729087114334106),
 ('декабрь', 0.8652557730674744),
 ('март', 0.8545795679092407),
 ('февраль', 0.8401415944099426)]

In [7]:
uk_emb.most_similar([uk_emb["серпень"]])

[('серпень', 0.9999998807907104),
 ('липень', 0.9096441268920898),
 ('вересень', 0.9016969203948975),
 ('червень', 0.8992518782615662),
 ('жовтень', 0.8810408115386963),
 ('листопад', 0.8787633180618286),
 ('квітень', 0.8592804670333862),
 ('грудень', 0.8586863279342651),
 ('травень', 0.840811014175415),
 ('лютий', 0.8256431221961975)]

In [8]:
ru_emb.most_similar([uk_emb["серпень"]])

[('Недопустимость', 0.24435284733772278),
 ('конструктивность', 0.23293082416057587),
 ('офор', 0.23256804049015045),
 ('deteydlya', 0.230317160487175),
 ('пресечении', 0.22632381319999695),
 ('одностороннего', 0.22608886659145355),
 ('подход', 0.2230587750673294),
 ('иболее', 0.22003726661205292),
 ('2015Александр', 0.21872766315937042),
 ('конструктивен', 0.21796567738056183)]

Load small dictionaries for corresponding word pairs as training set and test set.

In [9]:
def load_word_pairs(filename):
    uk_ru_pairs = []
    uk_vectors = []
    ru_vectors = []
    with open(filename, "r") as inpf:
        for line in inpf:
            uk, ru = line.rstrip().split("\t")
            if uk not in uk_emb or ru not in ru_emb:
                continue
            uk_ru_pairs.append((uk, ru))
            uk_vectors.append(uk_emb[uk])
            ru_vectors.append(ru_emb[ru])
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)

In [10]:
uk_ru_train, X_train, Y_train = load_word_pairs("/content/drive/MyDrive/ukr_rus.train.txt")

In [11]:
uk_ru_test, X_test, Y_test = load_word_pairs("/content/drive/MyDrive/ukr_rus.test.txt")

## Embedding space mapping

Let $x_i \in \mathrm{R}^d$ be the distributed representation of word $i$ in the source language, and $y_i \in \mathrm{R}^d$ is the vector representation of its translation. Our purpose is to learn such a linear transform $W$ that minimizes the Euclidean distance between $Wx_i$ and $y_i$ for some subset of word embeddings. Thus we can formulate the so-called Procrustes problem:

$$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$$
or
$$W^*= \arg\min_W ||WX - Y||_F$$

where $||*||_F$ is the Frobenius norm.

In Greek mythology, Procrustes or "the stretcher" was a rogue smith and bandit from Attica who attacked people by stretching them or cutting off their legs, so as to force them to fit the size of an iron bed. We do the same bad things with the source embedding space. Our Procrustean bed is the target embedding space.

![embedding_mapping.png](https://github.com/yandexdataschool/nlp_course/raw/master/resources/embedding_mapping.png)

![procrustes.png](https://github.com/yandexdataschool/nlp_course/raw/master/resources/procrustes.png)

But wait... $W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$ looks like simple multiple linear regression (without intercept fit). So let's code.

In [12]:
from sklearn.linear_model import LinearRegression

# YOUR CODE HERE
# Example: mapping = LinearRegression(fit_intercept=False).fit(X_train, Y_train)
mapping = LinearRegression(fit_intercept=False).fit(X_train, Y_train)

Let's take a look at neighbors of the vector of word _"серпень"_ (_"август"_ in Russian) after linear transform.

In [13]:
august = mapping.predict(uk_emb["серпень"].reshape(1, -1))
ru_emb.most_similar(august)

[('апрель', 0.8541286587715149),
 ('июнь', 0.8411202430725098),
 ('март', 0.839699387550354),
 ('сентябрь', 0.835986852645874),
 ('февраль', 0.8329297304153442),
 ('октябрь', 0.8311845660209656),
 ('ноябрь', 0.8278923630714417),
 ('июль', 0.8234528303146362),
 ('август', 0.8120501637458801),
 ('декабрь', 0.8039003610610962)]

We can see that the neighborhood of this embedding consists of different months, but the right variant is in ninth place.

As a quality measure we will use precision top-1, top-5 and top-10 (for each transformed Ukrainian embedding we count how many correct target pairs are found in the top N nearest neighbors in Russian embedding space).

In [14]:
def precision(pairs, mapped_vectors, topn=1):
    """
    :args:
        pairs = list of correct word pairs [(uk_word_0, ru_word_0), ...]
        mapped_vectors = list of embeddings after mapping from source embedding space to destination embedding space
        topn = the number of nearest neighbors in destination embedding space to choose from
    :returns:
        precision_val, float number, total number of words for which we can find correct translation in top K.
    """
    assert len(pairs) == len(mapped_vectors)
    num_matches = 0
    for i, (_, ru) in enumerate(pairs):
        # YOUR CODE HERE
        neighbours = ru_emb.most_similar(mapped_vectors[i], topn=topn)
        nearest_words = [word for word, _ in neighbours]
        if ru in nearest_words:
            num_matches += 1
    precision_val = num_matches / len(pairs)
    return precision_val


In [15]:
assert precision([("серпень", "август")], august, topn=5) == 0.0
assert precision([("серпень", "август")], august, topn=9) == 1.0
assert precision([("серпень", "август")], august, topn=10) == 1.0

In [16]:
assert precision(uk_ru_test, X_test) == 0.0
assert precision(uk_ru_test, Y_test) == 1.0

In [17]:
precision_top1 = precision(uk_ru_test, mapping.predict(X_test), 1)
precision_top5 = precision(uk_ru_test, mapping.predict(X_test), 5)

assert precision_top1 >= 0.635
assert precision_top5 >= 0.813

## Making it better (orthogonal Procrustean problem)

It can be shown (see original paper) that a self-consistent linear mapping between semantic spaces should be orthogonal.
We can restrict the transform $W$ to be orthogonal. Then we will solve the next problem:

$$W^*= \arg\min_W ||WX - Y||_F \text{, where: } W^TW = I$$

$$I \text{ - identity matrix}$$

Instead of making yet another regression problem, we can find the optimal orthogonal transformation using singular value decomposition. It turns out that the optimal transformation $W^*$ can be expressed via SVD components:
$$X^TY=U\Sigma V^T\text{, singular value decomposition}$$
$$W^*=UV^T$$

In [18]:
def learn_transform(X_train, Y_train):
    """
    :returns: W* : float matrix[emb_dim x emb_dim] as defined in formulae above
    """
    # YOUR CODE HERE
    scalar_product = np.dot(X_train.T, Y_train)
    U, sigma, V_t = np.linalg.svd(scalar_product)
    W = np.dot(U, V_t)
    return W

In [19]:
W = learn_transform(X_train, Y_train)

In [20]:
ru_emb.most_similar([np.matmul(uk_emb["серпень"], W)])

[('апрель', 0.8237906694412231),
 ('сентябрь', 0.8049713373184204),
 ('март', 0.8025653958320618),
 ('июнь', 0.8021842241287231),
 ('октябрь', 0.8001735806465149),
 ('ноябрь', 0.7934483289718628),
 ('февраль', 0.7914120554924011),
 ('июль', 0.7908109426498413),
 ('август', 0.7891016602516174),
 ('декабрь', 0.7686373591423035)]

In [21]:
assert precision(uk_ru_test, np.matmul(X_test, W)) >= 0.653
assert precision(uk_ru_test, np.matmul(X_test, W), 5) >= 0.824

## UK-RU Translator

Now we are ready to make a simple word-based translator: for each word in the source language in shared embedding space we find the nearest in the target language.


In [22]:
with open("/content/drive/MyDrive/fairy_tale.txt", "r") as inpf:
    uk_sentences = [line.rstrip().lower() for line in inpf]

In [23]:
def translate(sentence):
    """
    :args:
        sentence - sentence in Ukrainian (str)
    :returns:
        translation - sentence in Russian (str)

    * find Ukrainian embedding for each word in sentence
    * transform Ukrainian embedding vector
    * find nearest Russian word and replace
    """
    # YOUR CODE HERE
    if not sentence.strip():
        return sentence

    words = sentence.split()
    translated_words = []

    for word in words:
        if not word.isalpha():
            translated_words.append(word)
            continue

        if word not in uk_emb:
            translated_words.append(word)
            continue

        uk_vector = uk_emb[word]
        transformed_vector = np.matmul(uk_vector, W)

        nearest = ru_emb.most_similar([transformed_vector], topn=1)
        if nearest:
            ru_word = nearest[0][0]
            translated_words.append(ru_word)
        else:
            translated_words.append(word)

    return " ".join(translated_words)

In [24]:
assert translate(".") == "."
assert translate("1 , 3") == "1 , 3"
assert translate("кіт зловив мишу") == "кот поймал мышку"

In [25]:
for sentence in uk_sentences:
    print("src: {}\ndst: {}\n".format(sentence, translate(sentence)))

src: лисичка - сестричка і вовк - панібрат
dst: лисичка - сестричка и волк - панібрат

src: як була собі лисичка та зробила хатку, та й живе. а це приходять холоди. от лисичка замерзла та й побігла в село вогню добувать, щоб витопити. прибігає до одної баби та й каже:
dst: как была себе лисичка и сделала хатку, и и живе. а оно приходят холоди. из лисичка замерзла и и побежала во село огня добувать, чтобы витопити. прибегает к одной бабы и и каже:

src: — здорові були, бабусю! з неділею... позичте мені огню, я вам одслужу.
dst: — здоровые були, бабусю! со неділею... позичте мне огню, мной тебе одслужу.

src: — добре, — каже, — лисичко - сестричко. сідай погрійся трохи, поки я пиріжечки повибираю з печі!
dst: — добре, — каже, — лисичко - сестричко. садись погрійся трохи, пока мной пирожки повибираю со печі!

src: а баба макові пиріжки пекла. от баба вибирає пиріжки та на столі кладе, щоб прохололи; а лисичка підгляділа та за пиріг, та з хати... виїла мачок із середини, а туди напхала смі

Not so bad, right? We can easily improve translation using a language model and not one but several nearest neighbors in shared embedding space. But that's for next time.

## Would you like to learn more?

### Articles:
* [Exploiting Similarities among Languages for Machine Translation](https://arxiv.org/pdf/1309.4168)  - entry point for multilingual embedding studies by Tomas Mikolov (the author of W2V)
* [Offline bilingual word vectors, orthogonal transformations and the inverted softmax](https://arxiv.org/pdf/1702.03859) - orthogonal transform for unsupervised MT
* [Word Translation Without Parallel Data](https://arxiv.org/pdf/1710.04087)
* [Loss in Translation: Learning Bilingual Word Mapping with a Retrieval Criterion](https://arxiv.org/pdf/1804.07745)
* [Unsupervised Alignment of Embeddings with Wasserstein Procrustes](https://arxiv.org/pdf/1805.11222)

### Repos (with ready-to-use multilingual embeddings):
* https://github.com/facebookresearch/MUSE

* https://github.com/Babylonpartners/fastText_multilingual -